In [1]:
from streetview import search_panoramas
import folium
import numpy as np
from geopy.distance import geodesic
from sampled_already import sampled_already
from long_lat_multifamily_la2 import longitude_latitude_dict
import pandas as pd

In [2]:
class PanoramaGroupFinder:
    def __init__(self, latitude, longitude, search_grid_diameter_meters, search_grid_nxn, pano_ids_already_set):
        self.longitude = longitude
        self.latitude = latitude
        self.search_gridcell_meters = search_grid_diameter_meters//search_grid_nxn
        self.search_gridcell_nxn = search_grid_nxn
        self.pano_ids_already_set = pano_ids_already_set

    
    def find_nearby(self, lat, lon, lats_ls, lons_ls, panoid_ls):
      panos = search_panoramas(lat=lat, lon=lon)

      for pano in panos[:100]:
          lats_ls.append(pano.lat)
          lons_ls.append(pano.lon)
          panoid_ls.append(pano.pano_id)

      return lats_ls, lons_ls, panoid_ls

    
    def get_all_panos(self):
      lats_ls, lons_ls, panoid_ls = [], [], []


      north = geodesic(meters=self.search_gridcell_meters).destination((self.latitude, self.longitude), 0)
      east = geodesic(meters=self.search_gridcell_meters).destination((self.latitude, self.longitude), 90)
      # calculate grid cell sizes 
      search_grid_lat = north.latitude - self.latitude
      search_grid_lon = east.longitude - self.longitude
      

      for i in range(int(-self.search_gridcell_nxn//2), int(self.search_gridcell_nxn//2) + 1):
        for j in range(int(-self.search_gridcell_nxn//2), int(self.search_gridcell_nxn//2) + 1):
            lat = self.latitude + (i * search_grid_lat)
            lon = self.longitude + (j * search_grid_lon)
            lats_ls, lons_ls, panoid_ls = self.find_nearby(lat, lon, lats_ls, lons_ls, panoid_ls)
      
      return lats_ls, lons_ls, panoid_ls



     
    def get_groupings(self):
      lats_ls, lons_ls, panoid_ls = self.get_all_panos()
      print(f"found {len(panoid_ls)} different panos")
      # grouping_dict = self.create_groupings(lats_ls, lons_ls, panoid_ls)
    #   print(grouping_dict)
      # Create new lists to store filtered results
      filtered_panoid_ls = []
      filtered_lats_ls = []
      filtered_lons_ls = []
      
      # Iterate through all panoramas and only keep those not already in the set
      for i, panoid in enumerate(panoid_ls):
          if panoid not in self.pano_ids_already_set:
              filtered_panoid_ls.append(panoid)
              filtered_lats_ls.append(lats_ls[i])
              filtered_lons_ls.append(lons_ls[i])
      
      # Replace the original lists with the filtered ones
      # lats_ls, lons_ls, panoid_ls = filtered_lats_ls, filtered_lons_ls, filtered_panoid_ls

      return filtered_lats_ls, filtered_lons_ls, filtered_panoid_ls
      


      # Save grouping_dict to a Python file
      # with open('test_grouping_dict200_2.py', 'w') as f:
      #     f.write('grouping_dict = {\n')
      #     for key, value in grouping_dict.items():
      #         f.write(f"    '{key}': {value},\n")
      #     f.write('}')



In [5]:
# # Create an empty DataFrame with the required columns
# panos_df = pd.DataFrame(columns=['pano_id', 'lat', 'lon'])

# # Save the empty DataFrame to CSV
# csv_save_file = "panos.csv"
# panos_df.to_csv(csv_save_file, index=False)

# print(f"Created empty CSV file: {csv_save_file}")

Created empty CSV file: panos.csv


In [4]:
csv_save_file = "panos.csv"
panos_df = pd.read_csv(csv_save_file)

# This will be used to check for already sampled panoramas
pano_ids_already_set = set(panos_df['pano_id'])
# Get the number of panoramas already sampled
num_already_sampled = len(sampled_already)
num_locations_total = len(longitude_latitude_dict)





counter = 0
for coords in longitude_latitude_dict.values():
#   if counter > 10:
#       break
  
  
  lat = coords[0]
  lon = coords[1]


  if (lat, lon) in sampled_already:
      print("skip")
      continue
  
  
  counter += 1
  print(f"through {num_already_sampled} locations, {(num_already_sampled/num_locations_total * 100):.2f}% done")
  print(f"searching for panos around {lat, lon}")
  num_already_sampled += 1

  # Update sampled_already.py with the current coordinates

  lats_ls, lons_ls, panoid_ls = PanoramaGroupFinder(latitude = lat, longitude= lon, search_grid_diameter_meters = 200, search_grid_nxn = 2, pano_ids_already_set = pano_ids_already_set).get_groupings()
  
  for panoid in panoid_ls:
    # Add each new panorama ID to the set of already sampled panoramas
    pano_ids_already_set.add(panoid)

  # Create a temporary DataFrame with the new panorama data
  new_panos_df = pd.DataFrame({
      'pano_id': panoid_ls,
      'lat': lats_ls,
      'lon': lons_ls
  })
  
  # Append the new data to the existing DataFrame
  panos_df = pd.concat([panos_df, new_panos_df], ignore_index=True)
  
  # Save the updated DataFrame back to the CSV file
  panos_df.to_csv(csv_save_file, index=False)
  
  print(f"Added {len(panoid_ls)} new panoramas to {csv_save_file}")

  sampled_already.add((lat, lon))
  with open("sampled_already.py", "w") as f:
      f.write(f"sampled_already = {sampled_already}\n")

  





# checked_locations = sampled_already
# if         
#   return (lat, lon) in checked_locations


# PanoramaGroupFinder(latitude = 34.272, longitude= -118.484869, search_grid_diameter_meters = 200, search_grid_nxn = 2, grouping_radius_threshold = 5).get_groupings()



through 1 locations, 0.03% done
searching for panos around ((34.0411288, -118.4470615))
found 310 different panos
Added 0 new panoramas to panos.csv
through 2 locations, 0.07% done
searching for panos around ((34.0254277, -118.2973963))
found 393 different panos
Added 0 new panoramas to panos.csv
through 3 locations, 0.10% done
searching for panos around ((34.048576, -118.461807))
found 515 different panos
Added 0 new panoramas to panos.csv
through 4 locations, 0.14% done
searching for panos around ((34.1537785, -118.2428758))
found 516 different panos
Added 0 new panoramas to panos.csv
through 5 locations, 0.17% done
searching for panos around ((34.096716, -118.33647))
found 356 different panos
Added 0 new panoramas to panos.csv
through 6 locations, 0.21% done
searching for panos around ((34.026707, -118.4715787))
found 590 different panos
Added 0 new panoramas to panos.csv
through 7 locations, 0.24% done
searching for panos around ((34.024801, -118.3986724))
found 399 different panos